In [ ]:
#! pip install -U boto3 Pillow

In [1]:
import subprocess
cmd = "pip install -U keras==2.0 boto3 Pillow"
returned_value = subprocess.call(cmd, shell=True)

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['KERAS_BACKEND']='tensorflow'
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense


os.environ['CUDA_VISIBLE_DEVICES'] = '0,1' 

### Enable GPU Support ###
import tensorflow as tf

config = tf.ConfigProto(allow_soft_placement=True,log_device_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1
session = tf.Session(config=config)
import boto3
s3_client = boto3.client('s3',
                  endpoint_url='http://minio-10-1-190-92.nip.io',
                  aws_access_key_id='minio',
                  aws_secret_access_key='minio123')

Using TensorFlow backend.


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:1 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:3b:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:5e:00.0, compute capability: 6.1



In [4]:
def download_dir(prefix, local, bucket, client=s3_client):
    """
    params:
    - prefix: pattern to match in s3
    - local: local path to folder in which to place files
    - bucket: s3 bucket with target contents
    - client: initialized s3 client object
    """
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket':bucket ,
        'Prefix':prefix,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    for k in keys:
        dest_pathname = os.path.join(local, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        client.download_file(bucket, k, dest_pathname)

In [2]:
# load datasets
#download_dir(prefix='', local='./',bucket='cnn')
# Convolutional Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# Install Tensorflow from the website: https://www.tensorflow.org/versions/r0.12/get_started/os_setup.html

# Installing Keras
# pip install --upgrade keras

# Resolve OMP issue
# Error #15: Initializing libiomp5.dylib, but found libiomp5.dylib already initialized.


# Part 1 - Building the CNN

# Importing the Keras libraries and packages


# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
#classifier.add(Dense(output_dim = 128, activation = 'relu'))
#classifier.add(Dense(output_dim = 1, activation = 'sigmoid'))
classifier.add(Dense( 128, activation = 'relu'))
classifier.add(Dense(  1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 1,
                         validation_data = test_set,
                         nb_val_samples = 2000)
                         





Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Found 8000 images belonging to 2 classes.


/home/jovyan/.local/lib/python3.6/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., activation="relu")`
  '` call to the Keras 2 API: ' + signature)
/home/jovyan/.local/lib/python3.6/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  '` call to the Keras 2 API: ' + signature)


Found 2000 images belonging to 2 classes.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/jovyan/.local/lib/python3.6/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=250, epochs=1, validation_steps=2000)`
  '` call to the Keras 2 API: ' + signature)



Epoch 1/1



250/250 [==============================] - 310s - loss: 0.6487 - acc: 0.6184 - val_loss: 0.5917 - val_acc: 0.7120


In [11]:
## save model
classifier.save_weights("classifier.h5")
classifier.save("classifier")
saved_model = open('classifier', 'rb')

s3_client.put_object(Body=saved_model, Bucket="cnn", Key="classifier")

{'ResponseMetadata': {'RequestId': '1623A7712ABAC8FF',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'envoy',
   'date': 'Tue, 21 Jul 2020 03:39:57 GMT',
   'content-length': '0',
   'accept-ranges': 'bytes',
   'content-security-policy': 'block-all-mixed-content',
   'etag': '"1394aabd341434e022f745463453022c"',
   'vary': 'Origin',
   'x-amz-request-id': '1623A7712ABAC8FF',
   'x-xss-protection': '1; mode=block',
   'x-envoy-upstream-service-time': '104'},
  'RetryAttempts': 0},
 'ETag': '"1394aabd341434e022f745463453022c"'}